# Homework 5 - Visit the Wikipedia hyperlinks graph!
In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria. 

In [101]:
import pandas as pd
import json
import pickle
from tqdm import tqdm

from collections import defaultdict
from heapq import *
import numpy as np

## Research questions


### **[RQ1]** 
Build the graph <img src="https://latex.codecogs.com/gif.latex?G=(V,&space;E)" title="G=(V, E)" /> where *V* is the set of articles and *E* the hyperlinks among them, and provide its basic information:
 
- If it is direct or not
- The number of nodes
- The number of edges 
- The average node degree. Is the graph dense?

###### Build the graph!

In [3]:
F = open('wiki-topcats-reduced.txt','r') 
rows=F.read().split('\n') #split the rows
grafo={}#initialize the graph
reciver_nodes=set()
for row in rows:
        link=row.split('\t') 
        if link[0] not in grafo: #add the vertex if it doesn't exist
            try:
                grafo[link[0]]=set()
                grafo[link[0]].add(link[1]) #add the edge
                reciver_nodes.add(link[1])
            except: print('empty row')
        else:
            grafo[link[0]].add(link[1])
            reciver_nodes.add(link[1])
F.close()

empty row


In [18]:
onlyreciver=(reciver_nodes-set(grafo.keys()))
for node in onlyreciver:
    grafo[node]=set()

###### Find out if it's directed or not:

In [23]:
for neighbors in grafo['52']:
    print(grafo[neighbors])

{'1060341', '1061824', '402265', '402300', '630946', '1571179', '401457', '401227', '401137', '60219', '1062938', '1163407', '401067', '401053', '402715', '724192', '827334', '1163551', '1163338', '1288076', '824998', '776478', '1062323', '809904', '401505', '401171', '401315', '401018', '723911', '810461', '401975', '447882', '1061905', '401981', '401295', '595633', '1169888', '401310', '961942', '1061728', '1245651', '401184', '1058269', '1061885', '401154', '1184538', '401628', '1163806', '401474', '402718', '401019', '400980', '401231', '1161659', '1288276', '1394526', '1399606', '401609', '1184217', '946986', '167532', '606279'}
{'765915', '1062401', '263677', '751054', '823931', '806219', '1061960', '1400547', '8961', '1163407', '1164945', '1061048', '811181', '1707461', '1400548', '1061284', '1169028', '381238', '135342', '1408281', '1163579', '1164926', '1166302', '604445', '1164930', '1179784', '678211', '1383234', '1400452', '344394', '946977', '1163226', '1400461', '824192',

We see that it's directed, since we have unspecular edges.

###### Get the number of nodes!

In [24]:
nodes=len(grafo)
nodes

461194

###### Get the number of edges!

In [26]:
edges=0
for node in grafo:
    try: edges+=(len(grafo[node]))
    except: pass
edges

2645247

###### Get the average node degree. Is the graph dense?

In graph theory, the degree of a vertex of a graph is the number of edges incident to the vertex. The degree of a vertex $v$ is denoted $\deg(v)$.

In [27]:
avg_degree= edges/nodes
avg_degree

5.735649206190887

As we see, the average node degree is slightly great than six.
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges.
We can conclude that the graph is not dense. It is very sparse indeed.

In [41]:
F = open('wiki-topcats-page-names.txt','r')
articles={}
for line in F.readlines():
    num=line.split()[0]
    tit=line.split()[1:]
    title=' '.join(tit)
    articles[num]=title
F.close()

### **[RQ2]** 
Given a category <img src="https://latex.codecogs.com/gif.latex?C_0&space;=&space;\{article_1,&space;article_2,&space;\dots&space;\}" title="C_0 = \{article_1, article_2, \dots \}" /> as input we want to rank all of the nodes in *V* according to the following criteria:
	
* Obtain a *block-ranking*, where the blocks are represented by the categories. In particular, we want:

<img src="https://latex.codecogs.com/gif.latex?block_{RANKING}&space;=\begin{bmatrix}&space;C_0&space;\\&space;C_1&space;\\&space;\dots&space;\\&space;C_c\\&space;\end{bmatrix}" title="block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}" />
	
Each category $C_i$ corresponds to a list of nodes. 

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

<img src="https://latex.codecogs.com/gif.latex?$$distance(C_0,&space;C_i)&space;=&space;median(ShortestPath(C_0,&space;C_i))$$" title="distance(C_0, C_i) = median(ShortestPath(C_0, C_i))" />

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$  and $C_i$. Moreover, the length of a path is given by the sum of the weights of the edges it is composed by.



###### Creating the categories dictionary

We create a dictionary that associate evry category with his articles.

In [30]:
F = open('wiki-topcats-categories.txt','r')
categories={}
for line in F.readlines():
    riga=line.split(' ')
    categoria=(riga[0].replace('Category:','').replace(';',''))
    articles=(riga[1:-1])
    articles.append(riga[-1].replace('\n',''))
    categories[categoria]= articles
F.close()

We take care of categories that contains only articles that are not in our graph:

In [31]:
todelete=[]
for cat in categories:
    esiste=False
    for art in categories[cat]:
        esiste=art in grafo
        if esiste==True: break          
    if esiste==False:
        todelete.append(cat)
for cat in todelete:
    del categories[cat]

We remove every article that is not in our graph:

In [32]:
for cat in categories:
    toremove=[]
    for art in categories[cat]:
        if art not in grafo:
            toremove.append(art)
    for art in toremove:
        categories[cat].remove(art)

        

We store our final categories dictionary in an external pickle file so we can reload it in need.

In [34]:
with open('categories.pickle', 'wb') as handle:
    pickle.dump(categories, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [35]:
with open('categories.pickle', 'rb') as handle:
    categories = pickle.load(handle)

In [36]:
len(categories)

11989

###### The block ranking

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/DijkstraDemo.gif/220px-DijkstraDemo.gif)

In [130]:
def dijkstra(graph, f, t):
    #sorting the graph
    g = defaultdict(list, grafo)
    q= [(0,f)] #intializing the queque
    seen=set() #initializing the set of seen nodes
    distances = {f: 0} #initializing the dict of the distances
    while q:
        (cost,v1) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            if v1 == t: return (cost)
            if g.get(v1) is not None: 
                for v2 in g.get(v1):
                    if v2 in seen: continue
                    prev = distances.get(v2, None)
                    new = cost + 1
                    if prev is None or new < prev:
                        distances[v2] = new
                        heappush(q, (new, v2))

    return float("inf")


In [134]:
def catdistance(graph, catdict, cat1, cat2):
    cat1nodes= catdict[cat1]
    cat2nodes= catdict[cat2]
    distances=[]
    for node in cat1nodes:
        for node2 in cat2nodes:
            distances.append(dijkstra(graph=graph, f=node, t=node2))
    return np.median(distances)

In [135]:
catdistance(cat1='Japanese_rock_music_groups', cat2='Visual_kei_bands', catdict=categories, graph=grafo)

2.5

In [139]:
inputcat=input()

Japanese_rock_music_groups


In [166]:
blockranking=[(0, inputcat)]


In [167]:
blockranking

[(0, 'Japanese_rock_music_groups')]

In [168]:
count=0
for category in categories:
    blockranking.append((catdistance(cat1=inputcat, cat2=category,catdict=categories, graph=grafo), category))
    count+=1
    if count%10==0: print(count)

10
20
30
40
50
60
70
80


KeyboardInterrupt: 

In [169]:
sorted(blockranking)

[(0, 'Japanese_rock_music_groups'),
 (2.0, 'Virus-related_cutaneous_conditions'),
 (2.5, 'Japanese_rock_music_groups'),
 (2.5, 'Visual_kei_bands'),
 (3.0, 'Chemical_elements'),
 (3.0, 'Medical_emergencies'),
 (3.0, 'Oxidizing_agents'),
 (3.5, 'Alkaloids'),
 (3.5, 'Carboxylate_esters'),
 (3.5, 'Congenital_disorders'),
 (3.5, 'Reproduction'),
 (4.0, 'Abnormal_psychology'),
 (4.0, 'Alkenes'),
 (4.0, 'Greek_loanwords'),
 (4.0, 'Medical_treatments'),
 (4.0, 'Neurological_disorders'),
 (4.0, 'Oncology'),
 (4.0, 'Oxides'),
 (4.0, 'World_Health_Organization_essential_medicines'),
 (4.5, 'Autoimmune_diseases'),
 (4.5, 'Cold_War'),
 (4.5, 'Nutrition'),
 (4.5, 'Physiology'),
 (5.0, 'Bases_of_the_United_States_Air_Force'),
 (5.0, 'Companies_of_Germany'),
 (5.0, 'Disability'),
 (5.0, 'Lactones'),
 (5.0, 'Military_history_of_the_United_States_(19001999)'),
 (5.0, 'Military_units_and_formations_of_the_United_States_in_World_War_II'),
 (5.0, 'Neurology'),
 (5.0, 'Neurophysiology'),
 (5.0, 'Phenol_ethe

In [53]:
categories[inputcat]

['1756455',
 '1756475',
 '1756480',
 '1756486',
 '1756487',
 '1756490',
 '1756508',
 '1756510',
 '1756517',
 '1756528']

In [55]:
articles['1756480']

'Hameed Youssef'


* Once you obtain the $"block_{RANKING}"$ vector, you want to sort the nodes in each category. The way you should sort them is explained by this example:

	*	Suppose the categories order, given from the previous point, is <img src="https://latex.codecogs.com/gif.latex?C_0,&space;C_1,&space;C_2" title="C_0, C_1, C_2" />


__[STEP1]__ Compute subgraph induced by <img src="https://latex.codecogs.com/gif.latex?C_0" title="C_0" />. For each node compute the sum of the weigths of the in-edges.

 <img src="https://latex.codecogs.com/gif.latex?score_{article_i}&space;=&space;\sum_{i&space;\in&space;in-edges}&space;w_i" title="score_{article_i} = \sum_{j \in in-edges(article_i)} w_j" />

__[STEP2]__ Extend the graph to the nodes that belong to <img src="https://latex.codecogs.com/gif.latex?C_1" title="C_1" />. Thus, for each article in <img src="https://latex.codecogs.com/gif.latex?C_1" title="C_1" /> compute the score as before. __Note__ that the in-edges coming from the previous category, <img src="https://latex.codecogs.com/gif.latex?C_0" title="C_0" />, have as weights the score of the node that sends the edge.


__[STEP3]__ Repeat Step2 up to the last category of the ranking. In the last step of the example you clearly see the weight update of the edge coming from node *E*.
	
![alt text](imgs/algorithm.PNG)
